In [1]:
df = spark.read.csv('sales.csv', header=True, inferSchema=True)

In [6]:
df.select('Country').distinct().show()

+-------------+
|      Country|
+-------------+
|       Sweden|
|       Jersey|
|     Malaysia|
|       Turkey|
|      Germany|
|       France|
|      Belgium|
|      Finland|
|United States|
|        India|
|       Kuwait|
|        Malta|
|        Italy|
|       Norway|
|        Spain|
|      Denmark|
|      Ireland|
|       Israel|
|      Iceland|
|  South Korea|
+-------------+
only showing top 20 rows



In [9]:
df.filter("Country = 'Brazil' ").select("Name","Price").show()

+-------+-----+
|   Name|Price|
+-------+-----+
|Joachim| 1200|
|  Diana| 7500|
+-------+-----+



In [22]:
df.groupby("Country").sum('Price').withColumnRenamed('sum(Price)', 'TotalPrice').show()

+-------------+----------+
|      Country|TotalPrice|
+-------------+----------+
|       Sweden|      8400|
|       Jersey|      1200|
|     Malaysia|      1200|
|       Turkey|      2400|
|      Germany|     22800|
|       France|     30300|
|      Belgium|      3600|
|      Finland|      1200|
|United States|    350350|
|        India|      2400|
|       Kuwait|      1200|
|        Malta|      3600|
|        Italy|      2400|
|       Norway|     12000|
|        Spain|      2400|
|      Denmark|      8400|
|      Ireland|     29100|
|       Israel|      1200|
|      Iceland|      1200|
|  South Korea|      1200|
+-------------+----------+
only showing top 20 rows



In [16]:
df.groupby("Country").sum('Price').withColumnRenamed('sum(Price)', 'TotalPrice').orderBy('TotalPrice', ascending=False).show()

+--------------------+----------+
|             Country|TotalPrice|
+--------------------+----------+
|       United States|    350350|
|      United Kingdom|     63600|
|              Canada|     42000|
|              France|     30300|
|             Ireland|     29100|
|             Germany|     22800|
|           Australia|     22800|
|         Switzerland|     19200|
|         Netherlands|     14400|
|              Norway|     12000|
|              Brazil|      8700|
|             Denmark|      8400|
|              Sweden|      8400|
|             Belgium|      3600|
|        South Africa|      3600|
|               Malta|      3600|
|United Arab Emirates|      3600|
|             Austria|      3600|
|               Spain|      2400|
|              Turkey|      2400|
+--------------------+----------+
only showing top 20 rows



In [17]:
df2 = spark.read.csv('countries.csv', header=True, inferSchema=True)

In [19]:
df2.printSchema()

root
 |-- Country: string (nullable = true)
 |-- ID: integer (nullable = true)



In [24]:
tmp = df.groupby("Country").sum('Price').withColumnRenamed('sum(Price)', 'TotalPrice').orderBy('TotalPrice', ascending=False)
df2.join(tmp,'Country').select("ID","TotalPrice").show()

+---+----------+
| ID|TotalPrice|
+---+----------+
| 15|      8400|
| 29|      1200|
| 30|      1200|
| 24|      2400|
| 19|     22800|
|  5|     30300|
| 14|      3600|
| 11|      1200|
|  2|    350350|
|  9|      2400|
| 25|      1200|
| 26|      3600|
| 18|      2400|
| 16|     12000|
| 21|      2400|
| 13|      8400|
|  7|     29100|
|  4|      1200|
| 31|      1200|
| 32|      1200|
+---+----------+
only showing top 20 rows



In [125]:
from pyspark.sql.functions import *

numOfIterations = 10

lines = spark.read.text("pagerank_data.txt")
# You can also test your program on the follow larger data set:
# lines = spark.read.text("dblp.in.txt")

a = lines.select(split(lines[0],' '))
links = a.select(a[0][0].alias('src'), a[0][1].alias('dst'))
outdegrees = links.groupBy('src').count()
ranks = outdegrees.select('src', lit(1).alias('rank'))

from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
contrib = udf(lambda x,y: x/y, FloatType()) #for each neighbor, compute its contrib
update = udf(lambda x: 0.15 * 1 + 0.85 * x, FloatType()) #update formula

for iteration in range(numOfIterations):
    tmp = links.join(ranks,['src']).join(outdegrees,['src']) #schema: src - dst - rank of src - count of src
    cal_contrib = tmp.select('dst', contrib(tmp['rank'],tmp['count']).alias('contrib_to_dst')) #compute each neighbor's contri to dst
    groupby_contrib = cal_contrib.groupby('dst').sum('contrib_to_dst') #take the summation of right part of update formula
    ranks = groupby_contrib.select('dst',update(groupby_contrib['sum(contrib_to_dst)']).alias('rank')).withColumnRenamed('dst', 'src')
    

ranks.orderBy(desc('rank')).show()

+---+----------+
|src|      rank|
+---+----------+
|  1| 1.2981883|
|  3|       1.0|
|  4|       1.0|
|  2|0.70181173|
+---+----------+

